In [1]:
import pandas as pd
import numpy as np
import re
import json

In [2]:
pd.set_option('display.max_columns',50)

In [3]:
dataFile = './speciesdata.csv'

In [4]:
excludedCols = ['Responsável','Guia','Folder','Artigo','Família.1', 
                'Glossário','Fotos adultos','Fotos girinos',
                'Vídeos','Áudios', 'Diagnose','DOI' ]

df = pd.read_csv(dataFile, usecols=lambda x: x not in excludedCols)

In [5]:
# Drop every row after the one containing only null values
nullrow = df[df.apply(lambda x: all(x.isnull()), axis=1)].index[0]
df.drop(axis=1, index=range( nullrow,df.shape[0] ), inplace=True)

In [6]:
df.drop('Red List.1', axis=1, inplace=True)

In [7]:
df.rename( {'Espécie': 'speciesName', 
            'Família': 'family',
            'Postura dos ovos (habitat breeding)*': 'habitat_breeding',
            'Chances de encontros (muito raro/raro/frequente/muito frequente)': 'detectability',
            'Nome Comum': 'vernacularNames'}, axis=1, inplace=True)

In [8]:
df.replace('^-$', '', inplace=True, regex=True)

In [9]:
df

,family,speciesName,vernacularNames,Endêmico (Chapada),Endêmico (Cerrado),Red List,Habitat de vida (mata ou floresta/area aberta/pedras)*,habitat_breeding,Poleiro (tipical calling perch)*,Atividade (noturno/diurno),Tamanho real (mm) - fêmea,Tamanho real (mm) - macho,Meses de Ocorrência,detectability,Distribuição - SITE REUBER (espécies do DF),Ameaças - SITE REUBER (espécies do DF)
0,AROMOBATIDAE,"Allobates goianus (Bokermann, 1975)",,não,sim,NT,floresta (Fh),terrestre (Tr),Terrestre (Te),diurno,16.8-17,16.8-17,NaN,Muito raro,,
1,BUFONIDAE,"Rhaebo guttatus (Schneider, 1799)",Cururu de couro,não,não,LC,floresta (Fh),Lêntico (Le),Terrestre (Te),noturno,180,68,NaN,Raro,,
2,BUFONIDAE,"Rhinella mirandaribeiroi (Gallardo, 1965)",,não,sim,não avaliada,cerrado (Ce) e areas alteradas (Aa),Lêntico (Le),Terrestre (Te),noturno,49-73,40-71,NaN,Raro,,
3,BUFONIDAE,"Rhinella rubescens (Lutz, 1925)*",Cururu do cerrado,não,sim,LC,"areas abertas (Of), Cerrado (Ce), floresta (Fh...",Lêntico (Le) e Lótico (Lo),Terrestre (Te),noturno,NaN,NaN,ano todo,Frequente,Espécie com ampla distribuição associada ao bi...,Desmatamentos e remoção de hábitats constituem...
4,BUFONIDAE,"Rhinella schneideri (Werner, 1894)*",Sapo cururu,não,não,LC,"areas abertas (Of), Cerrado (Ce), floresta (Fh...",Lêntico (Le) e Lótico (Lo),Terrestre (Te),noturno,150-200,NaN,ano todo,Muito frequente,Espécie com ampla distribuição podendo ser enc...,Espécie não ameaçada.
5,CRAUGASTORIDAE,"Barycholos ternetzi (Miranda- Ribeiro, 1937)*",Rãzinha da chuva,não,sim,LC,"floresta (Fh), cerrado, (Ce)",terrestre (Tr),Terrestre (Te),diurno,NaN,23.1-29.4,chuva,Frequente,"Espécie restrita ao bioma Cerrado, pode ser en...",Desmatamento e remoção de hábitats constituem ...
6,DENDROBATIDAE,"Ameerega flavopicta (Lutz, 1925)*",Sapo flecha,não,sim,LC,"areas abertas (Of), Cerrado (Ce)",terrestre (Tr),Terrestre (Te),diurno,20-24,NaN,ano todo,Raro,Espécie com ampla distribuição no bioma Cerrad...,Desmatamentos e remoção de habitat.
7,HYLIDAE,"Aplastodiscus lutzorum (Berneck, Giaretta, Bra...",Perereca-verde-da-mata,não,sim,não avaliada,floresta (Fh),Lótico (Lo),Arbustos baixos (Hb) e arbustos altos (Lb),noturno,33.7,30.7–36,Chuva (Dezembro a Março),Raro,"Sua distribuição é no estado do DF e no Goiás,...",
8,HYLIDAE,"Bokermannohyla pseudopseudis (Miranda-Ribeiro,...",Falsa perereca,não,sim,LC,"areas abertas (Of), Cerrado (Ce) e floresta (Fh)",Lótico (Lo),Terrestre (Te) e arbustos baixos (Lb),noturno,60.2-61.6,51.9-65.3,chuva,Frequente,,
9,HYLIDAE,"Dendropsophus jimi (Napoli & Caramashi, 1999)",Pererequinha do Brejo,não,sim,LC,"areas abertas (Of), Cerrado (Ce)",Lêntico (Le),Grama (Gr),noturno,NaN,NaN,chuva,Frequente,Espécie associada ao bioma Cerrado. Ocorre nos...,NaN


### Species name, id and Authorship

In [10]:
df['scientificName'] = df['speciesName'].apply(lambda x: re.findall( '^\w+(?: cf.| aff.)? \w+\.?', str(x))[0])

In [11]:
df['scientificNameAuthorship'] = df[['speciesName','scientificName']].apply(lambda x: x[0][len(x[1]):], axis=1)
df['scientificNameAuthorship'] = df['scientificNameAuthorship'].apply(lambda x: re.findall( '[\w,\s\-&]+', str(x) )).apply(lambda x: ''.join(x).strip())

In [12]:
ids = df['scientificName'].str.replace('\s','-').str.replace('[.]','').str.lower()
df.insert(0, 'id', ids)

In [13]:
df.drop('speciesName', axis=1, inplace=True)

### Family name and id

In [14]:
df['family'] = df['family'].apply(lambda x: x.capitalize())
df['family_id'] = df['family'].str.lower()

### Vernacular name

In [15]:
def extractVernacularNames(string):
    names = re.split('/|;|,',string)
    names = [ n.strip().replace(' ','-').capitalize() for n in names ]
    if len(names)==1 and names[0]=='':
        return []
    else:
        return names

In [16]:
df['vernacularNames'] = df['vernacularNames'].apply(extractVernacularNames)

### Red list

In [17]:
df['Red List'] = df['Red List'].str.lower()
df.loc[ df['Red List']=='não avaliada','Red List']='ne'

In [18]:
df = pd.get_dummies(data=df, prefix='redlist', columns=['Red List'])

### Endemicidade

In [19]:
df['endemic_cerrado'] = df['Endêmico (Cerrado)'].apply(lambda x: 1 if x=='sim' else 0)
df['endemic_chapada'] = df['Endêmico (Chapada)'].apply(lambda x: 1 if x=='sim' else 0)
df.drop('Endêmico (Chapada)', axis=1, inplace=True)
df.drop('Endêmico (Cerrado)', axis=1, inplace=True)

In [20]:
# Helpers

getInsideParentheses = lambda x: [ str.lower(e) for e in re.findall( '\((.{0,5})\)', str(x) ) ]

### Detectability

In [21]:
df['detectability'] = df['detectability']\
    .str.replace('[^\w]','')\
    .str.replace('ê','e')\
    .str.lower()

In [22]:
subst_dict = {
    'muitofrequente': 'ff',
    'frequente':'f',
    'raro': 'r',
    'muitoraro': 'rr'
}

df['detectability'] = df['detectability'].apply(lambda x: subst_dict[x] if x is not np.NaN else x)
df = pd.get_dummies(df, prefix='detectability',columns=['detectability'])

### Poleiro

In [23]:
df['poleiro'] = df['Poleiro (tipical calling perch)* '].apply( getInsideParentheses )
df.drop('Poleiro (tipical calling perch)* ', axis=1, inplace=True)

In [24]:
poleiro_types = list(set( el for ls in df['poleiro'] for el in ls))

In [25]:
for tp in poleiro_types:
    df[f'tcp_{tp}'] = df['poleiro'].apply(lambda x: 1 if f'{tp}' in x else 0)

In [26]:
df.drop('poleiro', axis=1, inplace=True)

### Habitat breeding

In [27]:
matches = df['habitat_breeding'].str.lower().str.extractall('\((\w+)\)')
hb_dummies = pd.get_dummies(matches, prefix="habitat_breeding").groupby(level=0).sum()

In [28]:
hb_dummies = hb_dummies.reindex(df.index).fillna(0).astype(int)

In [29]:
df = pd.concat([df,hb_dummies], axis=1)

In [30]:
df.drop('habitat_breeding',axis=1, inplace=True)

### Habitat

In [31]:
df['habitat'] = df['Habitat de vida (mata ou floresta/area aberta/pedras)*'].apply( getInsideParentheses)
df.drop('Habitat de vida (mata ou floresta/area aberta/pedras)*', axis=1, inplace=True)

In [32]:
habitat_types = list(set( el for ls in df['habitat'] for el in ls ))

In [33]:
for tp in habitat_types:
    df[f'habitat_{tp}'] = df['habitat'].apply(lambda x: 1 if f'{tp}' in x else 0)

In [34]:
df.drop('habitat', axis=1, inplace=True)

### Atividade

In [35]:
df['Atividade (noturno/diurno)'].value_counts()

noturno              43
noturno e diurno      4
diurno                3
noturno e diurno      1
Name: Atividade (noturno/diurno), dtype: int64

In [36]:
df['atividade_diu'] = df['Atividade (noturno/diurno)'].apply(lambda x: 1 if 'diurno' in str(x) else 0)
df['atividade_not'] = df['Atividade (noturno/diurno)'].apply(lambda x: 1 if 'noturno' in str(x) else 0)
df.drop('Atividade (noturno/diurno)', axis=1, inplace=True)

In [37]:
df['Ameaças - SITE REUBER (espécies do DF)'][5]

'Desmatamento e remoção de hábitats constituem ameaças as populações locais.'

### Tamanho

In [38]:
df['tamanho_femea'] = df['Tamanho real (mm) - fêmea'].apply(lambda x: re.findall( '([0-9\.]+)' , str(x).replace(',','.')))
df['tamanho_macho'] = df['Tamanho real (mm) - macho'].apply(lambda x: re.findall( '([0-9\.]+)' , str(x).replace(',','.')))

In [39]:
df['tamanho_femea_med'] = df['tamanho_femea'].apply( lambda l: sum([float(i) for i in l])/len(l) if len(l)>0 else None )
df['tamanho_macho_med'] = df['tamanho_macho'].apply( lambda l: sum([float(i) for i in l])/len(l) if len(l)>0 else None )

In [40]:
#df['tamanho_femea_min'] = df['tamanho_femea'].apply( lambda x: min([ float(i) for i in x]) if len(x) > 0 else None)
#df['tamanho_femea_max'] = df['tamanho_femea'].apply( lambda x: max([ float(i) for i in x]) if len(x) > 0 else None)
#df['tamanho_macho_min'] = df['tamanho_macho'].apply( lambda x: min([ float(i) for i in x]) if len(x) > 0 else None)
#df['tamanho_macho_max'] = df['tamanho_macho'].apply( lambda x: max([ float(i) for i in x]) if len(x) > 0 else None)

#df['tamanho_macho_max'].replace(np.NaN, df['tamanho_femea_max'], inplace=True)
#df['tamanho_macho_min'].replace(np.NaN, df['tamanho_femea_min'], inplace=True)
#df['tamanho_femea_max'].replace(np.NaN, df['tamanho_macho_max'], inplace=True)
#df['tamanho_femea_min'].replace(np.NaN, df['tamanho_femea_min'], inplace=True)

In [41]:
df.drop('tamanho_femea', axis=1, inplace=True)
df.drop('tamanho_macho', axis=1, inplace=True)
df.drop('Tamanho real (mm) - fêmea', axis=1, inplace=True)
df.drop('Tamanho real (mm) - macho', axis=1, inplace=True)

In [42]:
df['tamanho_macho_med'].replace(np.NaN, df['tamanho_femea_med'], inplace=True)
df['tamanho_femea_med'].replace(np.NaN, df['tamanho_macho_med'], inplace=True)
df['tamanho_med'] = df[['tamanho_macho_med','tamanho_femea_med']].mean(axis=1)

### Temporarily remove fields

In [43]:
df.drop('Distribuição - SITE REUBER (espécies do DF)', axis=1, inplace=True)
df.drop('Ameaças - SITE REUBER (espécies do DF)', axis=1, inplace=True)

In [44]:
df

,id,family,vernacularNames,Meses de Ocorrência,scientificName,scientificNameAuthorship,family_id,redlist_dd,redlist_lc,redlist_ne,redlist_nt,endemic_cerrado,endemic_chapada,detectability_f,detectability_ff,detectability_r,detectability_rr,tcp_te,tcp_lb,tcp_tr,tcp_hb,tcp_gr,habitat_breeding_le,habitat_breeding_lo,habitat_breeding_tr,habitat_ce,habitat_of,habitat_fh,habitat_aa,atividade_diu,atividade_not,tamanho_femea_med,tamanho_macho_med,tamanho_med
0,allobates-goianus,Aromobatidae,[],NaN,Allobates goianus,"Bokermann, 1975",aromobatidae,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,16.90,16.90,16.900
1,rhaebo-guttatus,Bufonidae,[Cururu-de-couro],NaN,Rhaebo guttatus,"Schneider, 1799",bufonidae,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,180.00,68.00,124.000
2,rhinella-mirandaribeiroi,Bufonidae,[],NaN,Rhinella mirandaribeiroi,"Gallardo, 1965",bufonidae,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,61.00,55.50,58.250
3,rhinella-rubescens,Bufonidae,[Cururu-do-cerrado],ano todo,Rhinella rubescens,"Lutz, 1925",bufonidae,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,0,1,NaN,NaN,NaN
4,rhinella-schneideri,Bufonidae,[Sapo-cururu],ano todo,Rhinella schneideri,"Werner, 1894",bufonidae,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,1,1,1,1,0,1,175.00,175.00,175.000
5,barycholos-ternetzi,Craugastoridae,[Rãzinha-da-chuva],chuva,Barycholos ternetzi,"Miranda- Ribeiro, 1937",craugastoridae,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,1,0,26.25,26.25,26.250
6,ameerega-flavopicta,Dendrobatidae,[Sapo-flecha],ano todo,Ameerega flavopicta,"Lutz, 1925",dendrobatidae,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,1,1,0,0,1,0,22.00,22.00,22.000
7,aplastodiscus-lutzorum,Hylidae,[Perereca-verde-da-mata],Chuva (Dezembro a Março),Aplastodiscus lutzorum,"Berneck, Giaretta, Brandão, Cruz & Haddad, 2017",hylidae,0,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,33.70,33.35,33.525
8,bokermannohyla-pseudopseudis,Hylidae,[Falsa-perereca],chuva,Bokermannohyla pseudopseudis,"Miranda-Ribeiro, 1937",hylidae,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0,0,1,0,1,1,1,0,0,1,60.90,58.60,59.750
9,dendropsophus-jimi,Hylidae,[Pererequinha-do-brejo],chuva,Dendropsophus jimi,"Napoli & Caramashi, 1999",hylidae,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,1,NaN,NaN,NaN


### Write to json

In [45]:
d = df.to_dict(orient='records')

with open('datajson.json', 'w') as f:
    json.dump(d, f,indent=1, ensure_ascii=False)

In [46]:
with open('datajson.json', 'r') as f:
    spdata = json.load(f)